# Main - MongoDB

Per questa consegna è necessaria l'installazione di due pacchetti:
- pymongo
- mongoengine

1- Procediamo con l'installazione dei pacchetti:

In [17]:
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install mongoengine

%run Gestore.ipynb

2- Importiamo i pacchetti necessari al corretto funzionamento:

In [26]:
from pymongo import MongoClient

3 - Eseguiamo la funzione main:

In [27]:
if __name__ == '__main__':
    client = MongoClient('localhost', 27017)
    db = client.covid
    db_name = 'covid'
    gestore = Gestore(client, db)
    db_list = gestore.db_list()
    if db_name in db_list:
        print('Database', db_name, 'presente')
    else:
        print("Creazione database", db_name, "e aggiunta delle collezioni...")
        gestore.create_db(db_name)

Init effettuata
['admin', 'config', 'local', 'prova', 'test']
Creazione database covid e aggiunta delle collezioni...
creazione in corso...
DB e collezioni create correttamente


Esempi di query:
1- Numero di pazienti maschi positivi nella regione di Seoul

In [20]:
query = {'sex': 'male', 'region.province': 'Seoul'}
gestore.find_count(query, "patient_info")

Query:
 {'sex': 'male', 'region.province': 'Seoul'}
Numero di risultati: 
627
-------------------------------------------


È stato effettuato un text index sul campo "infection_case":
2- Recupera i primi 5 documenti dei pazienti di sesso femminile con infection_case: "contact" che sono stati dimessi nella provincia di Seoul

In [21]:
query = {'$and': [{'$text': {'$search': 'inflow'}, 'region.province':'Seoul', 'state': 'released', 'sex': 'female'}]}

gestore.findDb(query, "patient_info", False, 5)

Query:
 {'$and': [{'$text': {'$search': 'inflow'}, 'region.province': 'Seoul', 'state': 'released', 'sex': 'female'}]}
Risultati: 
[{'_id': ObjectId('5fd0f7bd2d75830f7ad9a76c'),
  'age': 7,
  'date_history': {'confirmed_date': datetime.datetime(2020, 5, 3, 0, 0),
                   'released_date': datetime.datetime(2020, 5, 30, 0, 0)},
  'infection_case': 'overseas inflow',
  'patient_id': '1600000044',
  'region': {'city': 'Jung-gu',
             'code': 10240,
             'elementary_school_count': 12,
             'kindergarten_count': 14,
             'province': 'Seoul',
             'university_count': 2},
  'sex': 'female',
  'state': 'released'},
 {'_id': ObjectId('5fd0f7bd2d75830f7ad9a760'),
  'age': 30,
  'date_history': {'confirmed_date': datetime.datetime(2020, 5, 12, 0, 0),
                   'released_date': datetime.datetime(2020, 5, 16, 0, 0)},
  'infection_case': 'overseas inflow',
  'patient_id': '1600000032',
  'region': {'city': 'Jung-gu',
             'code': 102

3- Recupera l'età media dei contagiati per provincia e ordina in senso crescente

In [28]:
pipeline = [
    {'$match': {"region.province": {'$exists':True}}},
    {'$group': {'_id': "$region.province", 'avgAge': {'$avg': "$age"}}},
    {'$project': {'roundedAge' : {'$round': ["$avgAge", 2]}}},
    {'$sort': {'roundedAge': 1}}
]

gestore.db_aggregate('patient_info', pipeline)

Query:
 [{'$match': {'region.province': {'$exists': True}}}, {'$group': {'_id': '$region.province', 'avgAge': {'$avg': '$age'}}}, {'$project': {'roundedAge': {'$round': ['$avgAge', 2]}}}, {'$sort': {'roundedAge': 1}}]
Risultati: 
[{'_id': 'Ulsan', 'roundedAge': 27.5},
 {'_id': 'Jeju-do', 'roundedAge': 44.5},
 {'_id': 'Daegu', 'roundedAge': 46.41},
 {'_id': 'Daejeon', 'roundedAge': 47.02},
 {'_id': 'Gangwon-do', 'roundedAge': 47.4},
 {'_id': 'Gyeongsangnam-do', 'roundedAge': 48.45},
 {'_id': 'Seoul', 'roundedAge': 48.95},
 {'_id': 'Gyeonggi-do', 'roundedAge': 49.08},
 {'_id': 'Chungcheongnam-do', 'roundedAge': 49.82},
 {'_id': 'Jeollabuk-do', 'roundedAge': 49.89},
 {'_id': 'Incheon', 'roundedAge': 50.33},
 {'_id': 'Sejong', 'roundedAge': 50.48},
 {'_id': 'Busan', 'roundedAge': 50.51},
 {'_id': 'Gyeongsangbuk-do', 'roundedAge': 50.62},
 {'_id': 'Chungcheongbuk-do', 'roundedAge': 54.3},
 {'_id': 'Jeollanam-do', 'roundedAge': 55.14}]
-------------------------------------------


4- Recupera tutte le persone che hanno contratto il virus e sono state positive per due settimane
   prima di essere dimesse.

In [29]:
pipeline = [
    {'$match': {"date_history.confirmed_date":{'$exists':True}, "date_history.released_date":{'$exists':True}}},
    {'$project':
        {'sex':1, 'age':1, 'region.province':1, 'region.city':1, "date_history.confirmed_date":1, "date_history.released_date":1, 'state':1,
            'eq': { '$cond': [ { '$eq': [{'$add': ["$date_history.confirmed_date", 1000 * 86400 * 14]}, '$date_history.released_date']}, 1, 0 ] }}},
    {'$match': {'eq':1}}

]
gestore.db_aggregate('patient_info', pipeline)

Query:
 [{'$match': {'date_history.confirmed_date': {'$exists': True}, 'date_history.released_date': {'$exists': True}}}, {'$project': {'sex': 1, 'age': 1, 'region.province': 1, 'region.city': 1, 'date_history.confirmed_date': 1, 'date_history.released_date': 1, 'state': 1, 'eq': {'$cond': [{'$eq': [{'$add': ['$date_history.confirmed_date', 1209600000]}, '$date_history.released_date']}, 1, 0]}}}, {'$match': {'eq': 1}}]
Risultati: 
[{'_id': ObjectId('5fd10f692d75830f7ad9b4f3'),
  'age': 72,
  'date_history': {'confirmed_date': datetime.datetime(2020, 2, 26, 0, 0),
                   'released_date': datetime.datetime(2020, 3, 11, 0, 0)},
  'eq': 1,
  'region': {'city': 'Jongno-gu', 'province': 'Seoul'},
  'sex': 'female',
  'state': 'released'},
 {'_id': ObjectId('5fd10f692d75830f7ad9b505'),
  'age': 42,
  'date_history': {'confirmed_date': datetime.datetime(2020, 2, 26, 0, 0),
                   'released_date': datetime.datetime(2020, 3, 11, 0, 0)},
  'eq': 1,
  'region': {'city': 'Ga

5- Conta le persone che sono morte dopo aver contratto il virus in gruppo e ordina in senso decrescente di morti.

In [30]:
pipeline = [
{'$match': {'infection_case':{'$exists':True}, 'group': True}},
{'$group': {'_id':'$infection_case'}},
{'$lookup': {
       'from': 'patient_info',
       'localField': '_id',
       'foreignField': 'infection_case',
       'as': 'patient'
     }
 },
  {'$unwind': "$patient"},
  {'$match': {"patient.state": "deceased"}},
  {'$group': {'_id':"$patient.infection_case", 'n_dead':{'$sum':1}}},
  {'$sort': {'n_dead':-1}}
]

gestore.db_aggregate('case', pipeline)

Query:
 [{'$match': {'infection_case': {'$exists': True}, 'group': True}}, {'$group': {'_id': '$infection_case'}}, {'$lookup': {'from': 'patient_info', 'localField': '_id', 'foreignField': 'infection_case', 'as': 'patient'}}, {'$unwind': '$patient'}, {'$match': {'patient.state': 'deceased'}}, {'$group': {'_id': '$patient.infection_case', 'n_dead': {'$sum': 1}}}, {'$sort': {'n_dead': -1}}]
Risultati: 
[{'_id': 'Cheongdo Daenam Hospital', 'n_dead': 7},
 {'_id': 'Shincheonji Church', 'n_dead': 2},
 {'_id': 'Gyeongsan Jeil Silver Town', 'n_dead': 1},
 {'_id': 'Guro-gu Call Center', 'n_dead': 1},
 {'_id': 'Day Care Center', 'n_dead': 1},
 {'_id': 'Bonghwa Pureun Nursing Home', 'n_dead': 1}]
-------------------------------------------


Metodo per eliminare un DB

In [25]:
gestore.delete_db(db_name)



['admin', 'config', 'covid', 'local', 'prova', 'test']
['admin', 'config', 'local', 'prova', 'test']
Database eliminato correttamente
